In [62]:
import json
from transformers import pipeline, set_seed

generator = pipeline("text-generation", model="gpt2")

def flatten_conversation(messages):
    prompt = ""
    for m in messages:
        if m["role"] == "user":
            prompt += f"User: {m['content']}\n"
        elif m["role"] == "assistant":
            prompt += f"Assistant: {m['content']}\n"
    if not messages or messages[-1]["role"] != "assistant":
        prompt += "Assistant:"
    return prompt

def run_single_turn(input_file, output_file):
    with open(input_file, "r") as f_in, open(output_file, "w") as f_out:
        for line in f_in:
            example = json.loads(line)
            instruction = example["instruction"]
            inp = example.get("input", "")
            prompt = instruction if not inp else f"{instruction}\n{inp}"

            generations = generator(
                prompt,
                max_new_tokens=100,
                num_return_sequences=1,
                do_sample=True,
                repetition_penalty=1.2,
                no_repeat_ngram_size=3,
                eos_token_id=generator.tokenizer.eos_token_id,
                pad_token_id=generator.tokenizer.eos_token_id
            )
            model_output = generations[0]["generated_text"][len(prompt):].strip()

            result = {
                "instruction": instruction,
                "input": inp,
                "reference_output": example.get("output", ""),
                "model_output": model_output
            }
            f_out.write(json.dumps(result) + "\n")

def run_multi_turn(input_file, output_file):
    with open(input_file, "r") as f_in, open(output_file, "w") as f_out:
        for line in f_in:
            example = json.loads(line)
            messages = example["messages"]

            prompt = flatten_conversation(messages[:-1])

            generations = generator(
                prompt,
                max_new_tokens=100,
                num_return_sequences=1,
                do_sample=True,
                repetition_penalty=1.2,
                no_repeat_ngram_size=3, 
                eos_token_id=generator.tokenizer.eos_token_id,
                pad_token_id=generator.tokenizer.eos_token_id
            )
            model_output = generations[0]["generated_text"][len(prompt):].strip()

            result = {
                "messages": messages,
                "reference_output": messages[-1]["content"],
                "model_output": model_output
            }
            f_out.write(json.dumps(result) + "\n")

run_single_turn("single_turn_test.jsonl", "baseline_single_turn_outputs.jsonl")
run_multi_turn("multi_turn_test.jsonl", "baseline_multi_turn_outputs.jsonl")

Device set to use cpu


In [63]:
from transformers import (
    pipeline, 
    set_seed, 
    GPT2Tokenizer, 
    GPT2LMHeadModel,
    Trainer,
    TrainingArguments,
    DataCollatorForLanguageModeling
)
from datasets import load_dataset, concatenate_datasets, DatasetDict
import torch

model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

def flatten_conversation(messages):
    prompt = ""
    for m in messages:
        if m["role"] == "user":
            prompt += f"User: {m['content']}\n"
        elif m["role"] == "assistant":
            prompt += f"Assistant: {m['content']}\n"
    if not messages or messages[-1]["role"] != "assistant":
        prompt += "Assistant:"
    return prompt


print("Loading datasets separately...")

# Note: We use split='train' even for a single file
st_train_ds = load_dataset('json', data_files="single_turn_train.jsonl", split="train")
mt_train_ds = load_dataset('json', data_files="multi_turn_train.jsonl", split="train")

st_val_ds = load_dataset('json', data_files="single_turn_val.jsonl", split="train")
mt_val_ds = load_dataset('json', data_files="multi_turn_val.jsonl", split="train")

print("Concatenating datasets...")
train_ds = concatenate_datasets([st_train_ds, mt_train_ds])
validation_ds = concatenate_datasets([st_val_ds, mt_val_ds])

datasets = DatasetDict({
    'train': train_ds,
    'validation': validation_ds
})

print(f"Loaded and combined dataset:")
print(datasets)
def preprocess_function(examples):
    prompts = []
    full_texts = []
    
    for i in range(len(examples['instruction'])):
        prompt_part = ""
        response_content = ""

        # Case 1: Multi-turn (messages is not None)
        if examples['messages'][i] is not None:
            messages = examples['messages'][i]
            if not messages: continue
            prompt_messages = messages[:-1]
            response_content = messages[-1]['content']
            prompt_part = flatten_conversation(prompt_messages)
        
        # Case 2: Single-turn (instruction is not None)
        elif examples['instruction'][i] is not None:
            instruction = examples['instruction'][i]
            inp = examples['input'][i]
            output = examples['output'][i]
            
            prompt_content = instruction
            if inp:
                prompt_content += f"\n{inp}"
            prompt_part = f"User: {prompt_content}\nAssistant: "
            response_content = output
        
        else:
            continue 
            
        full_text = prompt_part + response_content + tokenizer.eos_token
        prompts.append(prompt_part)
        full_texts.append(full_text)

    model_inputs = tokenizer(
        full_texts, max_length=256, truncation=True, padding="max_length"
    )
    prompt_token_lengths = [
        len(tokenizer(p, max_length=256, truncation=True)["input_ids"]) 
        for p in prompts
    ]
    labels = [list(row) for row in model_inputs["input_ids"]]
    for i in range(len(labels)):
        prompt_len = prompt_token_lengths[i]
        actual_prompt_len = min(prompt_len, len(labels[i]))
        labels[i][:actual_prompt_len] = [-100] * actual_prompt_len
    
    model_inputs["labels"] = labels
    return model_inputs

print("Tokenizing combined dataset...")
tokenized_datasets = datasets.map(
    preprocess_function,
    batched=True,
    remove_columns=datasets["train"].column_names 
)

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

training_args = TrainingArguments(
    output_dir="./gpt2-finetuned-combined",
    overwrite_output_dir=True,
    num_train_epochs=1.0,                   
    per_device_train_batch_size=4,          
    per_device_eval_batch_size=4,
    learning_rate=5e-5,                     
    weight_decay=0.01,
    logging_dir='./logs-combined',
    logging_steps=100,                      
    eval_strategy="steps",                  
    eval_steps=200,
    save_steps=200,                         
    save_total_limit=2,                     
    fp16=torch.cuda.is_available(),         
    report_to="none"  
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

print("Starting combined fine-tuning...")
trainer.train()

final_model_path = "./gpt2-finetuned-combined-final"
print(f"Training complete. Saving model to {final_model_path}")
trainer.save_model(final_model_path)
tokenizer.save_pretrained(final_model_path)

print("All done. 🎉")

Loading datasets separately...


Generating train split: 80 examples [00:00, 7266.01 examples/s]
Generating train split: 80 examples [00:00, 9990.01 examples/s]
Generating train split: 10 examples [00:00, 1998.33 examples/s]
Generating train split: 10 examples [00:00, 1998.33 examples/s]


Concatenating datasets...
Loaded and combined dataset:
DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output', 'constraints', 'evidence_ids', 'messages'],
        num_rows: 160
    })
    validation: Dataset({
        features: ['instruction', 'input', 'output', 'constraints', 'evidence_ids', 'messages'],
        num_rows: 20
    })
})
Tokenizing combined dataset...


Map: 100%|██████████| 20/20 [00:00<00:00, 832.76 examples/s]
C:\Users\luant\AppData\Local\Temp\ipykernel_19608\3508529908.py:128: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'pad_token_id': 50256}.


Starting combined fine-tuning...


c:\Users\luant\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss,Validation Loss


Training complete. Saving model to ./gpt2-finetuned-combined-final
All done. 🎉


In [ ]:
import torch
from transformers import pipeline, set_seed, GPT2Tokenizer

FINETUNED_MODEL_PATH = "./gpt2-finetuned-combined-final" 

print(f"Loading fine-tuned model from: {FINETUNED_MODEL_PATH}")
tokenizer = GPT2Tokenizer.from_pretrained(FINETUNED_MODEL_PATH)
generator = pipeline(
    "text-generation",
    model=FINETUNED_MODEL_PATH,
    tokenizer=tokenizer,
    device=0 if torch.cuda.is_available() else -1
)
print("Model loaded.")

def flatten_conversation(messages):
    prompt = ""
    for m in messages:
        if m["role"] == "user":
            prompt += f"User: {m['content']}\n"
        elif m["role"] == "assistant":
            prompt += f"Assistant: {m['content']}\n"
    if not messages or messages[-1]["role"] != "assistant":
        prompt += "Assistant:"
    return prompt

def run_single_turn_finetuned(input_file, output_file):
    print(f"Running single-turn inference on {input_file}...")
    with open(input_file, "r") as f_in, open(output_file, "w") as f_out:
        for line in f_in:
            example = json.loads(line)
            instruction = example["instruction"]
            inp = example.get("input", "")
            
            prompt_content = instruction if not inp else f"{instruction}\n{inp}"
            prompt = f"User: {prompt_content}\nAssistant:"

            generations = generator(
                prompt,
                max_new_tokens=100,
                num_return_sequences=1,
                do_sample=True,
                repetition_penalty=1.2,
                no_repeat_ngram_size=3,
                eos_token_id=tokenizer.eos_token_id, 
                pad_token_id=tokenizer.eos_token_id
            )
            
            model_output = generations[0]["generated_text"][len(prompt):].strip()

            result = {
                "instruction": instruction,
                "input": inp,
                "reference_output": example.get("output", ""),
                "model_output": model_output
            }
            f_out.write(json.dumps(result) + "\n")
    print(f"Single-turn results saved to {output_file}")

def run_multi_turn_finetuned(input_file, output_file):
    print(f"Running multi-turn inference on {input_file}...")
    with open(input_file, "r") as f_in, open(output_file, "w") as f_out:
        for line in f_in:
            example = json.loads(line)
            messages = example["messages"]
            prompt = flatten_conversation(messages[:-1])

            generations = generator(
                prompt,
                max_new_tokens=100,
                num_return_sequences=1,
                do_sample=True,
                repetition_penalty=1.2,
                no_repeat_ngram_size=3,
                eos_token_id=tokenizer.eos_token_id,
                pad_token_id=tokenizer.eos_token_id
            )
            model_output = generations[0]["generated_text"][len(prompt):].strip()

            result = {
                "messages": messages,
                "reference_output": messages[-1]["content"],
                "model_output": model_output
            }
            f_out.write(json.dumps(result) + "\n")
    print(f"Multi-turn results saved to {output_file}")

run_single_turn_finetuned(
    "single_turn_test.jsonl", 
    "combined_finetuned_single_turn_outputs.jsonl"
)

run_multi_turn_finetuned(
    "multi_turn_test.jsonl", 
    "combined_finetuned_multi_turn_outputs.jsonl"
)

print("\nAll fine-tuned (combined model) inference complete. ✅")


Device set to use cpu


Loading fine-tuned model from: ./gpt2-finetuned-combined-final
Model loaded.
Running single-turn inference on single_turn_test.jsonl...
Single-turn results saved to combined_finetuned_single_turn_outputs.jsonl
Running multi-turn inference on multi_turn_test.jsonl...
Multi-turn results saved to combined_finetuned_multi_turn_outputs.jsonl

All fine-tuned (combined model) inference complete. ✅


In [67]:
import json
import pandas as pd
from IPython.display import display, HTML

def compare_outputs(baseline_file, finetuned_file, num_examples=10):
    """
    Display side-by-side comparison of baseline vs fine-tuned outputs
    """
    baseline_data = []
    finetuned_data = []
    
    # Load baseline outputs
    with open(baseline_file, "r") as f:
        for line in f:
            baseline_data.append(json.loads(line))
    
    # Load fine-tuned outputs
    with open(finetuned_file, "r") as f:
        for line in f:
            finetuned_data.append(json.loads(line))
    
    # Create comparison dataframe
    comparisons = []
    for i in range(min(num_examples, len(baseline_data), len(finetuned_data))):
        baseline = baseline_data[i]
        finetuned = finetuned_data[i]
        
        # Handle single-turn format
        if "instruction" in baseline:
            instruction = baseline["instruction"]
            if baseline.get("input"):
                instruction += f"\nInput: {baseline['input']}"
        # Handle multi-turn format
        else:
            messages = baseline["messages"]
            instruction = "\n".join([f"{m['role']}: {m['content']}" for m in messages[:-1]])
        
        comparisons.append({
            "Example": i + 1,
            "Instruction": instruction,
            "Reference": baseline["reference_output"],
            "Baseline Output": baseline["model_output"],
            "Fine-tuned Output": finetuned["model_output"]
        })
    
    df = pd.DataFrame(comparisons)
    
    # Style the dataframe for better readability
    styled_df = df.style.set_properties(**{
        'text-align': 'left',
        'white-space': 'pre-wrap',
        'word-wrap': 'break-word',
        'max-width': '300px'
    }).set_table_styles([
        {'selector': 'th', 'props': [('text-align', 'center'), ('font-weight', 'bold')]},
        {'selector': 'td', 'props': [('padding', '10px'), ('border', '1px solid #ddd')]}
    ])
    
    return styled_df

def compare_outputs_simple(baseline_file, finetuned_file, num_examples=10):
    """
    Simple text-based comparison (no pandas styling)
    """
    baseline_data = []
    finetuned_data = []
    
    with open(baseline_file, "r") as f:
        for line in f:
            baseline_data.append(json.loads(line))
    
    with open(finetuned_file, "r") as f:
        for line in f:
            finetuned_data.append(json.loads(line))
    
    print("=" * 120)
    print(f"{'INSTRUCTION':<40} | {'BASELINE OUTPUT':<40} | {'FINE-TUNED OUTPUT':<40}")
    print("=" * 120)
    
    for i in range(min(num_examples, len(baseline_data), len(finetuned_data))):
        baseline = baseline_data[i]
        finetuned = finetuned_data[i]
        
        # Get instruction
        if "instruction" in baseline:
            instruction = baseline["instruction"]
            if baseline.get("input"):
                instruction += f" | {baseline['input']}"
        else:
            messages = baseline["messages"]
            instruction = messages[-2]["content"] if len(messages) > 1 else "Multi-turn"
        
        # Truncate long texts
        instruction = (instruction[:37] + "...") if len(instruction) > 40 else instruction
        baseline_out = (baseline["model_output"][:37] + "...") if len(baseline["model_output"]) > 40 else baseline["model_output"]
        finetuned_out = (finetuned["model_output"][:37] + "...") if len(finetuned["model_output"]) > 40 else finetuned["model_output"]
        
        print(f"\n{i+1}. {instruction:<38} | {baseline_out:<38} | {finetuned_out:<38}")
        print("-" * 120)


# If you want fancy pandas table (better for notebooks):
print("\n\n DETAILED SINGLE-TURN COMPARISON\n")
display(compare_outputs(
    "baseline_single_turn_outputs.jsonl",
    "combined_finetuned_single_turn_outputs.jsonl",
    num_examples=5
))

print("\n\n DETAILED MULTI-TURN COMPARISON\n")
display(compare_outputs(
    "baseline_multi_turn_outputs.jsonl",
    "combined_finetuned_multi_turn_outputs.jsonl",
    num_examples=5
))



 DETAILED SINGLE-TURN COMPARISON



,Example,Instruction,Reference,Baseline Output,Fine-tuned Output
0,1,Find four highly-rated gluten-free recipes with at least 5.0 stars and under 20 minutes.,1) Cottage-Cheese Stuffed Tomatoesâ€”10 min; 2) Nutritious Food Enhancerâ€”15 min; 3) Chili Lime BBQ Mangoesâ€”15 min; 4) No-Bake Low-Carb Protein Barsâ€”5 min,,"(1) Spicy Peanut Butter Banana Bread—7 min; 2), Gluten Free, Stuffed Rice & Garlic Oatmeal Cookies Made Easy With Scallion Muffins —2 1/4 hr 25mins; 3)—Chocolate Cucumber Cookie Doughnuts And Creamy Baked Sweet Potato Soup ★★★★ 4 g carbs • Serving Size : 30g protein ≥30 mg carbohydrates - including low sodium); 4.) Low Carb Chocolate Chip Brownies! #253529 |"
1,2,Find four high-protein lunchs over 34g protein in under 35 minutes.,1) Awesome Steamed Cheeseburgers!â€”46.0 g proteinâ€”16 min; 2) Vegetarian Sandwich with Herbed Cream Cheese and Guacamoleâ€”40.5 g proteinâ€”10 min; 3) Barbecued Buffalo Wing Dip With a Twistâ€”195.9 g proteinâ€”25 min; 4) Aussie Tuna Summer Saladâ€”47.4 g proteinâ€”16 min,"The other two meals are very low on carbs and fat, so the higher calorie versions will be your best bet for keeping you lean throughout this whole process! No dairy or gluten products included - just a good amount of plain flour & sugar (which is what we've been using). This works great as long it's not too hard to make enough extra filling out each day while eating breakfast cereal with some fruit juice etc... It also helps keep all dry ingredients down from being added later when there","1) Chunky Peanut Butter Chicken Soup—35 min; 2). Baked French Toast —45 min (no added sugar), served with a side of broccoli and tomatoes, bacon zest ($14)—30 min.; 3.) Kale Salad With Bacon Cheddar Cheese Sauce on the Side—10 min/d.—42 g carbs; 4] Eggnog Burger at Home Crab Steakhouse Chili Tacos & Tenders Pizza Pizzeria's BBQ Pork Pot Pie Sandwich""—25 h"
2,3,Find four low-carb dinners under 12g total carbohydrates with at least 34g protein.,1) Egyptian Red Snapper in Red Pepper Mint Sauceâ€”60.1 g protein; 2) Mustard-Grilled Scandinavian Salmonâ€”34.9 g protein; 3) Moroccan-Spiced Tunaâ€”53.4 g protein; 4) Melodee's Hot Wingsâ€”207.7 g protein,"This is the recipe for a pretty simple keto meal that you can make after your workout and still use in all of those workouts if needs be! I'm sure most people will have noticed, but there are other ingredients available which we'll discuss later on this post: 8 oz cans chickpeas or 1 cup canned pumpkin pie (I recommend using 2 cups) 30 g water 24 G sugar 16% fat 6 grams carbohydrate One serving x 14 servings Carbohydrates = 17 calories Serving Size Calories","1) Low Carb Chicken and Cheese—85 g carbs; 2). Glazed Cauliflower Soup (Spaghetti Squash)—56 mg carb; 3); Dessert Crusty Sweet & Sour Cream Pie With Honey Cheesecake Sauce in a Cupcake Mix—43 kcal, plus 13 min.; 4.) Apple Noodle Bowls Under 30 G Protein""—15 wk glycemic control or less; 5.). Chocolate Peanut Butter Rimmel Icecream Sandwich Bars Over 50–60 mL"
3,4,Find four highly-rated vegan recipes with at least 5.0 stars and under 30 minutes.,1) Tomato Lentil Stewâ€”25 min; 2) Vegan Waldorf Saladâ€”20 min; 3) Quick Elephant Earsâ€”10 min; 4) Summer Fruit & Vegetable Medleyâ€”18 min,"10) Chocolate Chip Cookies (Strawberry, Cream Cheese & Jelly), Vegan Eggplant Roll or Cookie Dough Tossed With Peanut Butter M&Ms – 4 Ingredients: 6 ounces of whole wheat flour 2 tablespoons coconut oil 1/2 teaspoon salt 3 teaspoons baking soda ¼ cup almond milk ½ tablespoon sugar Total Prep time 40 mins Cooking Time 20 min Preheat oven to 350 degrees F / 200°C Spray a 9×9 pan over medium heat; place all ingredients in the pot","1) Tomato Lime Pudding—6 min; 2!) Red Pepper Chicken Breast With Creamy Garlic Butter Sauce (Frozen)—4 min, Serves 3–8), Grilled Cheese Pot Pie & Desserts 434 mg sodium —1 g carbs •2.) Green Salad Mashed Potato Soup in Arugula Nuts""—5 hrs.;}3.—Grilling Salmon Steak Bowl on Peppers' Sesame Seeds—""13 cms"";]Daily Menu Items"" Da



 DETAILED MULTI-TURN COMPARISON



,Example,Instruction,Reference,Baseline Output,Fine-tuned Output
0,1,"user: I need lunch ideas. assistant: What's your time constraint and protein goal? user: Under 55 minutes, at least 17g.",1) Crispy Chicken Fingers with Spicy Honey Mustardâ€”62.8 g proteinâ€”25 min; 2) California Avocado Monte Cristoâ€”22.5 g proteinâ€”18 min; 3) Chicken Burrito Bundlesâ€”33.4 g proteinâ€”30 min; 4) Meatloaf Sandwiches Without the Loafâ€”34.2 g proteinâ€”40 min,How long do you want to be away from work for about 5 months before joining the company because of this job requirement?? Thanks For reading! Reply Delete,1) Spaghetti; 2—15 min —2 g carbs (from 1 egg); 3)—20 Min —26 mg sodium; 4–6 wk lactose free electrolytes available on site in 24-mL increments or under 32 mL.* This low carb version includes no added sugars except for the high potassium content of sorrels which can increase from 15 to 25 mmol/L if you have trouble digesting it.) Customer Service Representative: Are there any dietary restrictions with this program that
1,2,user: Find lunch ideas. assistant: Are you looking for low-carb options? user: Yes â€” under 12g carbs and at least 25g protein.,1) Crunchy Tuna Walnut Saladâ€”25.0 g proteinâ€”4.9 g carbs; 2) Whole Foods' Tarragon Chicken Saladâ€”26.1 g proteinâ€”8.0 g carbs; 3) Hackney's Inside-Out Burgerâ€”59.7 g proteinâ€”0.8 g carbs; 4) Appetizer or Snack Party Trayâ€”27.7 g proteinâ€”7.2 g carbs,"What are your ketogenic diet plans, especially if they include the carb restriction that is suggested by Dr Gossett in his book ""Fat Loss""? Do you have a plan to add weight or not limit carbohydrates during this period of time (eg when starting out)? Should we focus on calorie burn rather than caloric intake only as nutritionist advice while staying within our target range based upon nutritional facts about what works best with us/the body's needs etc? Is there any other option available which would be","1) Low Carb Baked Chicken Salad with Lemon Dressing—40 min; 2)—Chicken Fried With Grilled Cucumber Sauce (12 g carbohydrates); 3).5 mg sodium, 55 minutes; 4), Cornbread Omelet of Summer Rice Soup & Pasta by Olga Koczorowicz–Diazma in Italian Style or Sweet Potato Salsa Made Easy By Using A Vegan Tomato Paste! 1.) Carrot Pesto Quesadillas Deli Choco's"
2,3,"user: I need breakfast ideas. assistant: What's your time constraint and carbs goal? user: Under 55 minutes, under 24g.","1) Campbelled Eggsâ€”7.1 g carbsâ€”10 min; 2) Asparagus, Mushroom and Cheese Omelet With Herbsâ€”3.7 g carbsâ€”15 min; 3) Corsican Omeletteâ€”4.0 g carbsâ€”6 min; 4) Vegan Rice Puddingâ€”11.6 g carbsâ€”15 min",So why are you doing this over the past 3 days to get rid of all that carb consumption then?? You guys know what it does when we're not making a big commitment about food intake! It takes some getting used into conscious thought because if too much protein is absorbed or there was no appetite for carbohydrate at just 6 weeks old (or maybe even later?) those calories would be depleted as well...just like our children who will see their body take in less fat than they already have on average -,1) Broccoli—35 min; 2). Smoked Salmon —85 mL Fritos (30 g); 3)—Cucumber Salad with Bacon & Cream Sauce on the Pineapple Side.—20 h.; 4.) Peanut Butter Lemonade Sandwich at Home With Lime-Baked Cheese Balls in Tomato Basil Soup [MONDAYS]; 5)) Cajun Chicken Parmesan Pasta Cookies by Julia Childeau Style! Arugula Roasted Turkey Breast Wings Prepared By Lisa
3,4,"user: I want to make soup. assistant: Any dietary restrictions or sodium concerns? user: Yes, low sodium under 576 mg and vegetarian.",1) Zesty Mexican Tomato Soupâ€”519.1 mg sodium; 2) Garlic Stock for Soupsâ€”129.2 mg sodium; 3) Lebanese Adas Bis Silq (Lentil & Swisschard Soup)â€”218.4 mg sodium; 4) Moroccan Harira Soupâ€”182.0 mg sodium,"Anything else we can do for you this week that's just a little bit more flexible with what is going on in your diet right now..and how much of it will be consumed during th